In [5]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Modal analysis of a car speed dynamics

The dynamic matrix $A$ that represents the model is (see Example SS-09):

$$
A = \begin{bmatrix}-\frac{b}{m}&\frac{\tau_{\text{max}}\eta}{mr}\\0&-\frac{1}{2}\end{bmatrix},
$$

where $m$ is the car's mass, $r$ the wheel radius, $b$ the drag coefficient, $\eta$ the gear ratio and $\tau_{\text{max}}$ the maximum motor torque. The system's state is $x=[v, \tau_{\text{%}}]^T$ where $v$ is the velocity and $\tau_{\text{%}}$ is the % of maximum torque applied. Due to the particular structure of $A$ (upper triangular) the eigenvalues are $-\frac{b}{m}$ and $-\frac{1}{2}$ thus, since $b\ge0$ and $m>0$, the system is asymptotically stable for all parameter values.

It's worth noting that when $b=\frac{m}{2}$ the $A$ matrix assumes a form equal to a Jordan block scaled by a factor $\frac{\tau_{\text{max}}\eta}{mr}$ so the system has one eigenvalue with algebraic multiplicity 2 and geometric multiplicity 1.

### How to use this notebook?

- Verify the dependence of the first eigenvalue by only $m$ and $b$ by changing the parameters value.
- Try to achieve $b=\frac{m}{2}$ and check the resulting modes.

In [6]:
#Preparatory Cell 
import control
import numpy
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib inline

#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [7]:
#define the sliders for m, k and c
m = widgets.FloatSlider(
    value=1000,
    min=400,
    max=2000,
    step=1,
    description='m [kg]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
eta = widgets.FloatSlider(
    value=4,
    min=0.8,
    max=10.0,
    step=0.1,
    description='$\eta$:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
tau_max = widgets.FloatSlider(
    value=150,
    min=50,
    max=900,
    step=1,
    description=r'$\tau_{\text{max}}$ [Nm]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
b_air = widgets.FloatSlider(
    value=60,
    min=0,
    max=800,
    step=1,
    description=r'$b$ [Ns/m]:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [8]:
#function that make all the computations
def main_callback(m, eta, tau_max, b_air):
    eig1 = -b_air/m
    eig2 = -1/2
    
    if numpy.real([eig1,eig2])[0] == 0 and numpy.real([eig1,eig2])[1] == 0:
        T = numpy.linspace(0,20,1000)
    else:
        if min(numpy.abs(numpy.real([eig1,eig2]))) != 0:
            T = numpy.linspace(0,7*1/min(numpy.abs(numpy.real([eig1,eig2]))),1000)
        else:
            T = numpy.linspace(0,7*1/max(numpy.abs(numpy.real([eig1,eig2]))),1000)
    if eig1 == eig2:
        mode1 = numpy.exp(eig1*T)
        mode2 = T*mode1
    else:
        mode1 = numpy.exp(eig1*T)
        mode2 = numpy.exp(eig2*T)
    
    fig = plt.figure(figsize=[16, 5])
    fig.set_label('Modes')
    g1 = fig.add_subplot(121)
    g2 = fig.add_subplot(122)
    
    g1.plot(T,mode1)
    g1.grid()
    g1.set_xlabel('Time [s]')
    g1.set_ylabel('First mode')
    
    g2.plot(T,mode2)
    g2.grid()
    g2.set_xlabel('Time [s]')
    g2.set_ylabel('Second mode')
    
    # print('The eigenvalues are: -%.3f+%.3fj -%.3f-%.3fj' %(abs(eig1.real),abs(eig1.imag),abs(eig2.real),abs(eig2.imag)))
    modesString = 'The eigenvalues are $' + str(numpy.around(eig1,decimals=3)) + '$ and $' + str(numpy.around(eig2,decimals=3)) + '$ '
    if eig1 == eig2:
        modesString = modesString + 'with the corresponding modes $e^{' + str(numpy.around(eig1,decimals=3))\
                        + ' t}$ and $te^{' + str(numpy.around(eig2,decimals=3)) + ' t}$.'
    else:
        modesString = modesString + 'with the corresponding modes $e^{' + str(numpy.around(eig1,decimals=3))\
                        + ' t}$ and $e^{' + str(numpy.around(eig2,decimals=3)) + ' t}$.'
    display(Markdown(modesString))

        
out = widgets.interactive_output(main_callback,{'m':m,'eta':eta,'tau_max':tau_max,'b_air':b_air})
sliders = widgets.VBox([widgets.HBox([m,eta]),
                        widgets.HBox([tau_max,b_air])])
display(out,sliders)

Output()